#  Aula 11: Machine Learning (Cont...)

A base de dados 20 Newsgroups é uma coleção de, aproximadamente, 20 mil documentos com textos de notícias, distribuídas igualmente entre 20 tipos de tópicos.

Alguns dos tópicos são bastante similares como, por exemplo, comp.sys.ibm.pc.hardware e comp.sys.mac.hardware. Outros são bem destintos (misc.forsale e soc.religion.christian). Abaixo segue a lista completa dos 20 tópicos

``comp.graphics
comp.os.ms-windows.misc
comp.sys.ibm.pc.hardware
comp.sys.mac.hardware
comp.windows.x	
rec.autos
rec.motorcycles
rec.sport.baseball
rec.sport.hockey	
sci.crypt
sci.electronics
sci.med
sci.space
misc.forsale	
talk.politics.misc
talk.politics.guns
talk.politics.mideast	
talk.religion.misc
alt.atheism
soc.religion.christian``

Como exemplo, podemos ver abaixo um desses textos:

### comp.sys.mac.hardware

From: narlochn@kirk.msoe.edu

Subject: General questions on software and hardware...

Distribution: usa

Organization: Milwaukee School Of Engineering, Milwaukee, WI USA

Lines: 18

I have just been introduced to Macintosh systems.
I have a few questions.  E-mail prefered.

1) Just what is System 7?  I want hard details not dingy commercials like their ads in magazines...

2) Has anyone used the Microsoft Office 3.0?  I would like suggestions on, and descriptions on:

* each has a microsoft before the actual name.

a) *Word 5.1:

b) *Excel 4.0:

c) *Power Point:

d) *Mail 3.1

3)  What is the major differences between Mac Wordperfect and Word?

Thanks a lot!


### rec.sport.baseball

From: zappala@pollux.usc.edu (Daniel Zappala)

Subject: Angels win!

Article-I.D.: pollux.1psvouINNa2l

Distribution: world

Organization: University of Southern California, Los Angeles, CA

Lines: 6

NNTP-Posting-Host: pollux.usc.edu


The Angels won their home opener against the Brewers today before 33,000+ 
at Anaheim Stadium, 3-1 on a 3-hitter by Mark Langston.  J.T. Snow and 
Gary Discarcina hit home runs for the Angels.

Daniel

### Objetivo:

Montar um modelo de Machine learning para prever em qual dos 20 grupos um novo texto pertence. 

In [1]:
import json
import logging
import pickle
import shutil
import re
import codecs
import csv
import os, errno
import tarfile
import time
import sys
import numpy as np
import pandas as pd
import scipy.sparse as sp
from pprint import pprint
from random import shuffle
import unicodedata

In [2]:
texto = "bla Bla Bla 995378129"
t = re.compile(r'[0-9]')
t.findall(texto)

['9', '9', '5', '3', '7', '8', '1', '2', '9']

In [22]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_files
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
import joblib

In [3]:
from itertools import islice

In [11]:
cache = {"train": {'target': [], 'DESCR': [], 'target_names': [], 'data': [], 'filenames': []} , 
         "test": {'target': [], 'DESCR': [], 'target_names': [], 'data': [], 'filenames': []}}

In [12]:
len(cache)

2

In [13]:
data_train = cache['train']
data_test = cache['test']

In [14]:
print(list(data_train))
print("\n")
print(list(data_test))

['target', 'DESCR', 'target_names', 'data', 'filenames']


['target', 'DESCR', 'target_names', 'data', 'filenames']


In [15]:
data_train["target"]

[]

In [16]:
data_train["target_names"]

[]

In [17]:
data_train["data"]

[]

In [18]:
data_train["filenames"]

[]

Podemos ler os arquivos a partir de duas pastas. Uma contendo arquivos para treinamento e outra contendo os arquivos para a validação do nosso modelo.

In [19]:
data_home = "/Users/gustavorsampaio/Documents/Cursos/Python 2022.2/Aulas/20news-bydate/"
TRAIN_FOLDER = "/Users/gustavorsampaio/Documents/Cursos/Python 2022.2/Aulas/20news-bydate/20news-bydate-train"
TEST_FOLDER = "/Users/gustavorsampaio/Documents/Cursos/Python 2022.2/Aulas/20news-bydate/20news-bydate-test"

In [20]:
TRAIN_FOLDER

'/Users/gustavorsampaio/Documents/Cursos/Python 2022.2/Aulas/20news-bydate/20news-bydate-train'

In [23]:
list(load_files(TRAIN_FOLDER))

['data', 'filenames', 'target_names', 'target', 'DESCR']

In [24]:
paulo = {"train":load_files(TRAIN_FOLDER), "test":load_files(TEST_FOLDER)}

In [26]:
cache = dict(train=load_files(TRAIN_FOLDER, encoding='latin1'), test=load_files(TEST_FOLDER, encoding='latin1'))

In [27]:
len(cache)

2

In [28]:
list(cache)

['train', 'test']

In [29]:
list(cache["train"])

['data', 'filenames', 'target_names', 'target', 'DESCR']

In [30]:
cache["train"]["data"][10]

'From: maler@vercors.imag.fr (Oded Maler)\nSubject: Re: FLAME and a Jewish home in Palestine\nNntp-Posting-Host: pelvoux\nOrganization: IMAG, University of Grenoble, France\nLines: 40\n\nIn article <C5HJBC.1HC@bony1.bony.com>, jake@bony1.bony.com (Jake Livni) writes:\n|> In article <1993Apr13.172422.2407@newshub.ariel.yorku.ca> nabil@ariel.yorku.ca (Nabil Gangi) writes:\n|> \n|> >According to Exodus, there were 600,000 Jews that marched out of Egypt.\n|> \n|> This is only the number of adult males.  The total number of Jewish\n|> slaves leaving Egypt was much larger.\n|> \n|> >The number which could have arrived to the Holy Lands must have been\n|> >substantially less ude to the harsh desert and the killings between the\n|> >Jewish tribes on the way..\n|> >\n|> >NABIL\n|> \n|> Typical Arabic thinking.  If we are guilty of something, so is\n|> everyone else.  Unfortunately for you, Nabil, Jewish tribes are not\n|> nearly as susceptible to the fratricidal murdering that is still so\n|> c

In [31]:
cache["train"]["target_names"]

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [32]:
cache["train"]["target"][10]

17

In [33]:
data_train = cache['train']
data_test = cache['test']

In [34]:
print(list(data_train))
print("\n")
print(list(data_test))

['data', 'filenames', 'target_names', 'target', 'DESCR']


['data', 'filenames', 'target_names', 'target', 'DESCR']


In [35]:
data_train["data"][10]

'From: maler@vercors.imag.fr (Oded Maler)\nSubject: Re: FLAME and a Jewish home in Palestine\nNntp-Posting-Host: pelvoux\nOrganization: IMAG, University of Grenoble, France\nLines: 40\n\nIn article <C5HJBC.1HC@bony1.bony.com>, jake@bony1.bony.com (Jake Livni) writes:\n|> In article <1993Apr13.172422.2407@newshub.ariel.yorku.ca> nabil@ariel.yorku.ca (Nabil Gangi) writes:\n|> \n|> >According to Exodus, there were 600,000 Jews that marched out of Egypt.\n|> \n|> This is only the number of adult males.  The total number of Jewish\n|> slaves leaving Egypt was much larger.\n|> \n|> >The number which could have arrived to the Holy Lands must have been\n|> >substantially less ude to the harsh desert and the killings between the\n|> >Jewish tribes on the way..\n|> >\n|> >NABIL\n|> \n|> Typical Arabic thinking.  If we are guilty of something, so is\n|> everyone else.  Unfortunately for you, Nabil, Jewish tribes are not\n|> nearly as susceptible to the fratricidal murdering that is still so\n|> c

In [36]:
data_train["data"][0]

"From: cubbie@garnet.berkeley.edu (                               )\nSubject: Re: Cubs behind Marlins? How?\nArticle-I.D.: agate.1pt592$f9a\nOrganization: University of California, Berkeley\nLines: 12\nNNTP-Posting-Host: garnet.berkeley.edu\n\n\ngajarsky@pilot.njin.net writes:\n\nmorgan and guzman will have era's 1 run higher than last year, and\n the cubs will be idiots and not pitch harkey as much as hibbard.\n castillo won't be good (i think he's a stud pitcher)\n\n       This season so far, Morgan and Guzman helped to lead the Cubs\n       at top in ERA, even better than THE rotation at Atlanta.\n       Cubs ERA at 0.056 while Braves at 0.059. We know it is early\n       in the season, we Cubs fans have learned how to enjoy the\n       short triumph while it is still there.\n"

In [37]:
data_train["filenames"][0]

'/Users/gustavorsampaio/Documents/Cursos/Python 2022.2/Aulas/20news-bydate/20news-bydate-train/rec.sport.baseball/102736'

### Limpeza da base

Devemos realizar uma limpeza nos textos para eliminar quebra de linhas, tabs, ...

Tanto na base de treinamento:

In [38]:
data_train.data = [w.replace("\r\n", " ") for w in data_train.data]
data_train.data = [w.replace("\t", "") for w in data_train.data]
data_train.data = [w.replace("\n", "") for w in data_train.data]
data_train.data = [w.replace(">", "") for w in data_train.data]
data_train.data = [w.replace("<", "") for w in data_train.data]

In [39]:
data_train["data"][0]

"From: cubbie@garnet.berkeley.edu (                               )Subject: Re: Cubs behind Marlins? How?Article-I.D.: agate.1pt592$f9aOrganization: University of California, BerkeleyLines: 12NNTP-Posting-Host: garnet.berkeley.edugajarsky@pilot.njin.net writes:morgan and guzman will have era's 1 run higher than last year, and the cubs will be idiots and not pitch harkey as much as hibbard. castillo won't be good (i think he's a stud pitcher)       This season so far, Morgan and Guzman helped to lead the Cubs       at top in ERA, even better than THE rotation at Atlanta.       Cubs ERA at 0.056 while Braves at 0.059. We know it is early       in the season, we Cubs fans have learned how to enjoy the       short triumph while it is still there."

In [40]:
data_train["data"][10]

'From: maler@vercors.imag.fr (Oded Maler)Subject: Re: FLAME and a Jewish home in PalestineNntp-Posting-Host: pelvouxOrganization: IMAG, University of Grenoble, FranceLines: 40In article C5HJBC.1HC@bony1.bony.com, jake@bony1.bony.com (Jake Livni) writes:| In article 1993Apr13.172422.2407@newshub.ariel.yorku.ca nabil@ariel.yorku.ca (Nabil Gangi) writes:| | According to Exodus, there were 600,000 Jews that marched out of Egypt.| | This is only the number of adult males.  The total number of Jewish| slaves leaving Egypt was much larger.| | The number which could have arrived to the Holy Lands must have been| substantially less ude to the harsh desert and the killings between the| Jewish tribes on the way..| | NABIL| | Typical Arabic thinking.  If we are guilty of something, so is| everyone else.  Unfortunately for you, Nabil, Jewish tribes are not| nearly as susceptible to the fratricidal murdering that is still so| common among Arabs in the Middle East.  There were no " killings| between 

quanto na base de validação:

In [41]:
data_test.data = [w.replace("\r\n", " ") for w in data_test.data]
data_test.data = [w.replace("\t", "") for w in data_test.data]
data_test.data = [w.replace("\n", "") for w in data_test.data]

### Descrever os dados:

In [42]:
print('Existem nesta base de treinamento ' + str(len(data_train.target_names)) + ' classes:\n')
print("Elas São ... \n")
print('\t', data_train.target_names, '\n')

Existem nesta base de treinamento 20 classes:

Elas São ... 

	 ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc'] 



In [43]:
data_train["data"][0]

"From: cubbie@garnet.berkeley.edu (                               )Subject: Re: Cubs behind Marlins? How?Article-I.D.: agate.1pt592$f9aOrganization: University of California, BerkeleyLines: 12NNTP-Posting-Host: garnet.berkeley.edugajarsky@pilot.njin.net writes:morgan and guzman will have era's 1 run higher than last year, and the cubs will be idiots and not pitch harkey as much as hibbard. castillo won't be good (i think he's a stud pitcher)       This season so far, Morgan and Guzman helped to lead the Cubs       at top in ERA, even better than THE rotation at Atlanta.       Cubs ERA at 0.056 while Braves at 0.059. We know it is early       in the season, we Cubs fans have learned how to enjoy the       short triumph while it is still there."

In [44]:
data_train.data[100]

'From: sera@zuma.UUCP (Serdar Argic)Subject: Role of \'SDPA.ORG\' in slaughter of Gunduz, Ariyak, Arikan, Benler,...Reply-To: sera@zuma.UUCP (Serdar Argic)Distribution: worldLines: 107In article 1993Apr16.044001.15540@urartu.sdpa.org hla@urartu.sdpa.org  writes:Sure it is. It tells us how far right the whole Turkish political spectrum Nobody ever exposed your crimes like that before? What was your personal role in the murder of Orhan Gunduz and Kemal Arikan, again? How many moreMuslims will be slaughtered by \'SDPA.ORG\' as publicly declared and filedwith legal authorities? Please spell it out for us. "...that more people have to die..."                     SDPA 91@urartu.UUCP  "Yes, I stated this and stand by it."                    SDPA 255@urartu.UUCP    January 28, 1982 - Los AngelesKemal Arikan is slaughtered by two Armenians while driving to work.     March 22, 1982 - Cambridge, MassachusettsPrelude to grisly murder. A gift and import shop belonging toOrhan Gunduz is blown up. Gu

### Processando os textos para estimação

Extrair palavras do texto e converter para vetores numéricos:

max_df: Eliminar palavras nos documentos que aparecam com uma frequência muito elevada. Elas não irão adicionar muito ao nosso modelo.

In [45]:
count_vect = CountVectorizer(max_df=0.75) 
X_train_counts = count_vect.fit_transform(data_train.data)

In [46]:
X_train_counts

<11314x210310 sparse matrix of type '<class 'numpy.int64'>'
	with 1728698 stored elements in Compressed Sparse Row format>

In [47]:
X_train_counts.shape

(11314, 210310)

In [48]:
print('Nos ' + str(X_train_counts.shape[0]) + ' processos de treinamento temos ' + str(X_train_counts.shape[1]) + ' palavras')

Nos 11314 processos de treinamento temos 210310 palavras


Listar Palavras que mais aparecem

In [49]:
list(islice(count_vect.vocabulary_.items(), 20))

[('cubbie', 69862),
 ('garnet', 92339),
 ('berkeley', 52009),
 ('edu', 79458),
 ('re', 157006),
 ('cubs', 69874),
 ('behind', 51346),
 ('marlins', 124966),
 ('how', 102030),
 ('article', 45867),
 ('agate', 38594),
 ('1pt592', 10810),
 ('f9aorganization', 85389),
 ('university', 193049),
 ('california', 58827),
 ('berkeleylines', 52014),
 ('12nntp', 5302),
 ('posting', 150186),
 ('host', 101868),
 ('edugajarsky', 79714)]

Imaginem que textos mais longos possuem mais palavras e, desta maneira, teriam maior peso.

Para eliminar tal problema, iremos, ao invés de utilizar número de palavras, utilizar a sua frequência dentro de um texto

TfidfTransformer: Transformas uma matriz de contagem em uma matriz de frequências.

``Transform a count matrix to a normalized tf or tf-idf representation. Tf means term-frequency while tf-idf means term-frequency times inverse document-frequency``

In [50]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [51]:
X_train_tfidf.shape

(11314, 210310)

Listar Palavras que mais aparecem com maior detalhes

In [52]:
count_vect = CountVectorizer(stop_words='english', max_df=0.75, ngram_range=(1, 3))

In [53]:
X_train_counts = count_vect.fit_transform(data_train.data)

In [54]:
X_train_counts.shape

(11314, 2741169)

In [55]:
list(islice(count_vect.vocabulary_.items(), 50))

[('cubbie', 748673),
 ('garnet', 1093758),
 ('berkeley', 468461),
 ('edu', 904406),
 ('cubs', 748772),
 ('marlins', 1561272),
 ('article', 380815),
 ('agate', 302083),
 ('1pt592', 86695),
 ('f9aorganization', 993133),
 ('university', 2521966),
 ('california', 551965),
 ('berkeleylines', 468792),
 ('12nntp', 38576),
 ('posting', 1894821),
 ('host', 1233071),
 ('edugajarsky', 916173),
 ('pilot', 1853949),
 ('njin', 1703881),
 ('net', 1685277),
 ('writes', 2683390),
 ('morgan', 1638693),
 ('guzman', 1166581),
 ('era', 952884),
 ('run', 2103089),
 ('higher', 1212863),
 ('year', 2716043),
 ('idiots', 1260599),
 ('pitch', 1856168),
 ('harkey', 1183678),
 ('hibbard', 1210812),
 ('castillo', 578407),
 ('won', 2662760),
 ('good', 1129773),
 ('think', 2419433),
 ('stud', 2303926),
 ('pitcher', 1856396),
 ('season', 2153396),
 ('far', 1006517),
 ('helped', 1205377),
 ('lead', 1443315),
 ('better', 472317),
 ('rotation', 2096148),
 ('atlanta', 404739),
 ('056', 11470),
 ('braves', 514550),
 ('059'

In [57]:
count_vect1 = CountVectorizer(ngram_range=(1, 3))
X_train_counts1 = count_vect1.fit_transform(data_train.data)
list(islice(count_vect1.vocabulary_.items(), 20))

[('from', 1216443),
 ('cubbie', 880696),
 ('garnet', 1255429),
 ('berkeley', 598899),
 ('edu', 1024851),
 ('subject', 2674276),
 ('re', 2371131),
 ('cubs', 880795),
 ('behind', 588962),
 ('marlins', 1810891),
 ('how', 1429360),
 ('article', 473807),
 ('agate', 305942),
 ('1pt592', 85160),
 ('f9aorganization', 1115709),
 ('university', 3091079),
 ('of', 2028197),
 ('california', 698218),
 ('berkeleylines', 599228),
 ('12nntp', 37981)]

Podemos construir um DataFrame com Palavras que mais aparecem

Função ravel():

``x = np.array([[1, 2, 3], [4, 5, 6]])

np.ravel(x)

array([1, 2, 3, 4, 5, 6])``

In [58]:
x = np.array([[1, 2, 3], [4, 5, 6]])

In [59]:
x

array([[1, 2, 3],
       [4, 5, 6]])

In [60]:
np.ravel(x)

array([1, 2, 3, 4, 5, 6])

In [61]:
occ = np.asarray(X_train_counts.sum(axis=0)).ravel().tolist()

In [62]:
len(occ)

2741169

In [64]:
count_vect

CountVectorizer(max_df=0.75, ngram_range=(1, 3), stop_words='english')

In [66]:
counts_df = pd.DataFrame({'term': count_vect.get_feature_names_out(), 'occurrences': occ})
counts_df.sort_values(by='occurrences', ascending=False).head(20)

,term,occurrences
425054,ax,62089
426032,ax ax,56047
426307,ax ax ax,50430
904406,edu,16792
652440,com,10195
2683390,writes,7833
380815,article,6970
863017,don,5787
1826754,people,5524
1468757,like,5496


Listar Palavras que mais aparecem com maior detalhes por frequência pesando tamanho do texto

In [67]:
tfidf_transformer = TfidfTransformer(use_idf=False) # Enable inverse-document-frequency reweighting
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(11314, 2741169)

Dataframe com Palavras que mais aparecem

In [68]:
weights = np.asarray(X_train_tfidf.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': count_vect.get_feature_names_out(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(20)

,term,weight
904406,edu,0.066391
652440,com,0.039626
2683390,writes,0.030184
380815,article,0.026204
1894821,posting,0.023643
1233071,host,0.022187
1895191,posting host,0.021693
2521966,university,0.019331
863017,don,0.018489
1468757,like,0.018357


### Construir primeiro modelo de ML

In [69]:
text_clf = Pipeline([('vect', CountVectorizer(stop_words='english', max_df=0.75, ngram_range=(1, 3))),
                    ('tfidf', TfidfTransformer(use_idf=False)),
                    ('clf', MultinomialNB(fit_prior=False)), #Whether to learn class prior probabilities or not. If false, a uniform prior will be used.
                    ])

In [70]:
text_clf

Pipeline(steps=[('vect',
                 CountVectorizer(max_df=0.75, ngram_range=(1, 3),
                                 stop_words='english')),
                ('tfidf', TfidfTransformer(use_idf=False)),
                ('clf', MultinomialNB(fit_prior=False))])

In [71]:
text_clf_ok = text_clf.fit(data_train.data, data_train.target)

In [72]:
text_clf_ok

Pipeline(steps=[('vect',
                 CountVectorizer(max_df=0.75, ngram_range=(1, 3),
                                 stop_words='english')),
                ('tfidf', TfidfTransformer(use_idf=False)),
                ('clf', MultinomialNB(fit_prior=False))])

In [73]:
data_test.data[0:3]

['From: stimpy@dev-null.phys.psu.edu (Gregory Nagy)Subject: Re: ESPN UP YOURS .........Organization: Penn State Laboratory for Elementary Steam PhysicsLines: 52NNTP-Posting-Host: dev-null.phys.psu.eduIn article <C5u542.3CD@news.udel.edu> tmavor@earthview.cms.udel.edu writes:>>>>[Various justifiable rantings on ESPN coverage by several deleted]>>>>The only way to change ESPN\'s thinking, if it is even possible, is to complain>to them directly.  Anyone know there telephone # in Bristol, Ct?  Heh... Try the rec.autos.sport FAQ. They are always calling ESPN to complain.I\'m sure you could find the number for ABC there too, as many west-coast viewers were compaining about how something as boring as hockey cut intothe Long Beach GP. =)>>I do find it hard to believe that ESPN doesn\'t think viewers will simply>change the channel from a boring game....I know I did.  And then, when>they didn\'t show the NYI-Wash overtime(s), I was livid!  If I wanted>to watch baseball, I could have turned on th

In [74]:
predictedNB = text_clf_ok.predict(data_test.data)

In [75]:
predictedNB

array([10, 16, 10, ..., 13,  3,  0])

In [76]:
data_test.target

array([10, 16, 14, ...,  4,  6,  7])

In [77]:
predictedNB == data_test.target

array([ True,  True, False, ..., False, False, False])

In [78]:
np.mean(predictedNB == data_test.target)

0.7837227827934148

In [79]:
data_test.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [80]:
data_test.data[497]

'From: zstern@adobe.com (Zalman Stern)Subject: Re: Adobe Photo Shop type software for Unix/X/Motif platforms?Organization: Adobe Systems IncorporatedLines: 24Charles Boesel writes> > In article <C5w8xB.Iv6@world.std.com>  (sci.image.processing,comp.graphics), wdm@world.std.com (Wayne Michael)  writes:> >    I have been searching for a quality image enhancement and> >    manipulation package for Unix/X/Motif platforms that is comparable> >    to Adobe Photo Shop for the Mac. [stuff deleted]> > I understand that Adobe is working on making Photoshop available for> the SGI Indigo, but that is just "rumor" and I wouldn\'t bet on it> until I see it. But they >are< going to release Illustrator for the SGI> "real soon now."> Illustrator for SGI is a shipping product. Adobe and SGI have announced that  Photoshop is being ported to SGI machines. A simillar announcement has been  made by Adobe and Sun for Sun platforms. No dates have been announced to the  best of my knowledge.--Zalman Stern   za

In [81]:
data_test.target[497]

1

In [82]:
data_test.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [83]:
text_clf_ok.predict([data_test.data[497]])

array([1])

In [94]:
luana = "you are preparing for an exam thursday. The exam will be very easy and you can use \
you mac computer to do it. while you are doing you exams I will be watching the world \
cup games. Soccer games and baseball games as well"

In [95]:
luana

'you are preparing for an exam thursday. The exam will be very easy and you can use you mac computer to do it. while you are doing you exams I will be watching the world cup games. Soccer games and baseball games as well'

In [96]:
text_clf_ok.predict([luana])

array([10])

In [97]:
data_test.target_names[9]

'rec.sport.baseball'

In [99]:
andre = "From: jbrown@vax.cns.muskingum.eduSubject: ESPN cares less about hockeyOrganization: Muskingum CollegeLines: 23ESPN is pathetic, I have been watching everybody complain about ESPN's coverageand I agree with all of you.  Might I suggest that we are getting all worked upover nothing.  We ALL knew that ESPN's coverage was going to suck.  I mean haveyou ever watched during the regular season Sportscenter coverage of hockey? Itsucks, I mean really sucks!!!  ESPN does not know hockey if it shot them in theass.  I was really pissed of about last night, coverage.  I go to school in OhioI *know* the Indians suck, so why show them?  If I were Bill Clement I wouldshoot myself.Anyway after I calmed down after I realized that they were not going to showovertime of the Caps Isles game I thought to myself I knew that this was goingto happen, in fact I remerbed myself praying that at least they would keep eventhe boring Devils game.Overall I think ESPN does not care if they show hockey or not and I firmlybelive that they have know respect for the fans. ie *because of contract       obligations*, screw you ESPN.Praise be to all hockey fans who put up with this shit!GO HAWKS!!!JB"

In [100]:
text_clf_ok.predict([andre])

array([10])

In [101]:
data_test.target_names[10]

'rec.sport.hockey'

In [102]:
data_test.data[5000]

'From: jmeritt@mental.mitre.orgSubject: Rawlins has been listening to the DevilOrganization: UTexas Mail-to-News GatewayLines: 5NNTP-Posting-Host: cs.utexas.eduGod ItSelf appeared to me and spoke to me, saying "Rawlins has been listening toa deamon, and has been taken in by its satanic words!"Now, how we tell which divine inspiration comes from the One True God and whichcomes from a satanic trickster?'

In [103]:
data_test.target[5000]

19

In [104]:
data_test.target_names[19]

'talk.religion.misc'

In [105]:
#texto = 'From: stimpy@dev-null.phys.psu.edu (Gregory Nagy)Subject: Re: ESPN UP YOURS .........Organization: Penn State Laboratory for Elementary Steam PhysicsLines: 52NNTP-Posting-Host: dev-null.phys.psu.eduIn article <C5u542.3CD@news.udel.edu> tmavor@earthview.cms.udel.edu writes:>>>>[Various justifiable rantings on ESPN coverage by several deleted]>>>>The only way to change ESPN\'s thinking, if it is even possible, is to complain>to them directly.  Anyone know there telephone # in Bristol, Ct?  Heh... Try the rec.autos.sport FAQ. They are always calling ESPN to complain.I\'m sure you could find the number for ABC there too, as many west-coast viewers were compaining about how something as boring as hockey cut intothe Long Beach GP. =)>>I do find it hard to believe that ESPN doesn\'t think viewers will simply>change the channel from a boring game....I know I did.  And then, when>they didn\'t show the NYI-Wash overtime(s), I was livid!  If I wanted>to watch baseball, I could have turned on the Phillies-Padres extra>inning game....instead, I went to bed angry......I boycotted ESPN\'s>morning Sportscenter today, I was still so incensed.Were you (and several of the other people here it seems) asleep the day"contracts" were explained? ASPN has a piece of paper saying it MUSTshow that baseball game if it happens. Many businesses payedd money tohave their commercials run during a baseball game. This is a business,not your own personal video servant.>>My wife says I shouldn\'t go to bed angry, but last nite.........GRRRRRRR!>Maybe you should put that anger into something positive. For example, I sawads for the new Dodge both on the ESPN and KBL broadcasts. Why not write toDodge saying that "thanks to the ads run during the STANLEY CUP PLAYOFFS, you will now concider their products in the future. They love to hear stufflike that and in the future will be more willing to buy commercial timefor hockey games, giving ESPN (and other networks) more incentive to carrygames (just one example)Come on people, as great as we think it is, Hockey does not leapfrog the"big three" overight.> >--------------------------------------------------------------------->Tim Mavor   |  "I am known by many names.......>College of Marine Studies  |   some call me.........Tim.">Univ. of Delaware   |    >Newark, DE 19716   |  "You know much that is hidden, O\' Tim!">tmavor@pandora.cms.udel.edu|  Monty Python and the Holy Grail--------------------------------------------------------------------->'
#texto1 = 'One very good player played in the national baseball team'
texto = 'From: JEK@cu.nih.gov Subject: Mary, "blessed among women" Lines: 23  Dave Bernard writes:   > When Elizabeth greeted Mary with the words: "Blessed art thou  > among women" (Luke 1:42), it appears that this places Mary  > beyond the sanctification of normal humanity.  But Deborah says (Judges 5:24):   > Blessed among women shall be Jael the wife of Heber the Kenite,  > Blessed above all women in the tents.  It can doubtless be taken that Jael\'s slaying of Sisera was a type of Mary\'s victory over sin. But even if we take Deborah\'s words as applying prophetically or symbolically to Mary, they must still be applicable literally to Jael.  We may well take them to mean that God used her as a part of His plan for the deliverance of His people, and that she has this in common with Mary.  But we have no reason to suppose that they mean that she was sinless, and thus no reason to take the like expression applied to Mary as proof that she was sinless.   Yours,  James Kiefer'

In [106]:
texto1 = 'Python is a very cool language. We can use it to program computers to do that we want. We can also break the computer motherboard if we perform very fast actions. Also, we can collect sports information from the web. If we collect the data with a Machintoch computer things will be much faster. Macs are the best computers in the market. Mac computers do not break. Apple is a very strong company'

In [107]:
predictedNB = text_clf_ok.predict([texto])

In [108]:
predictedNB

array([15])

In [109]:
data_test.target_names[15]

'soc.religion.christian'

In [111]:
data_test.target_names[11]

'sci.crypt'

In [112]:
text_clf_ok.predict_proba([luana])

array([[0.02320331, 0.04508057, 0.04298924, 0.04333396, 0.070177  ,
        0.03411242, 0.07078026, 0.03182282, 0.02688393, 0.14179184,
        0.16814726, 0.04514707, 0.0509542 , 0.0370633 , 0.03002339,
        0.03291536, 0.03110471, 0.02669439, 0.02539111, 0.02238386]])

### Construir segundo modelo de ML 

In [113]:
text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')),
                        ('tfidf', TfidfTransformer()),
                        ('clf', MultinomialNB(fit_prior=False)),
])

parameters = {'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
              'vect__max_df': (0.75, 0.80),
              'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2, 1e-5), # Additive (Laplace/Lidstone) smoothing parameter
}

In [114]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1) # Number of jobs to run in parallel. None means 1. -1 means using all processors

In [115]:
gs_clf

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(stop_words='english')),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultinomialNB(fit_prior=False))]),
             n_jobs=-1,
             param_grid={'clf__alpha': (0.01, 1e-05),
                         'tfidf__use_idf': (True, False),
                         'vect__max_df': (0.75, 0.8),
                         'vect__ngram_range': [(1, 1), (1, 2), (1, 3)]})

In [116]:
gs_clf_ok = gs_clf.fit(data_train.data, data_train.target)

/Users/gustavorsampaio/miniconda3/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [117]:
gs_clf.best_score_

0.9086975615492842

In [118]:
gs_clf.best_params_

{'clf__alpha': 0.01,
 'tfidf__use_idf': False,
 'vect__max_df': 0.75,
 'vect__ngram_range': (1, 1)}

In [119]:
text_clf_final = Pipeline([('vect', CountVectorizer(stop_words='english', max_df=0.75, ngram_range=(1, 1))),
                    ('tfidf', TfidfTransformer(use_idf=False)),
                    ('clf', MultinomialNB(fit_prior=False, alpha=0.01)), #Whether to learn class prior probabilities or not. If false, a uniform prior will be used.
                    ])

In [120]:
text_clf_final

Pipeline(steps=[('vect', CountVectorizer(max_df=0.75, stop_words='english')),
                ('tfidf', TfidfTransformer(use_idf=False)),
                ('clf', MultinomialNB(alpha=0.01, fit_prior=False))])

In [121]:
predicted = gs_clf.predict(data_test.data)
print('Taxa de acerto: ', np.mean(predicted == data_test.target))

Taxa de acerto:  0.8345724907063197


### Construir terceiro modelo de ML 

In [122]:
text_clf_svm = Pipeline([('vect', CountVectorizer(stop_words='english')),
                     ('tfidf', TfidfTransformer()),
                     ('clf-svm', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, max_iter=15, random_state=42)), # loss: The loss function to be used, penalty: The penalty (aka regularization term) to be used, n_iter: The number of passes over the training data, random_state: The seed of the pseudo random number generator to use when shuffling the data
])
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
                  'vect__max_df': (0.75, 0.80),
                  'tfidf__use_idf': (True, False),
                  'clf-svm__alpha': (1e-2, 1e-5),
}

In [123]:
gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)

In [124]:
gs_clf_svm_ok = gs_clf_svm.fit(data_train.data, data_train.target)

/Users/gustavorsampaio/miniconda3/lib/python3.11/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [125]:
gs_clf_svm_ok.best_score_

0.9228390597522205

In [126]:
gs_clf_svm_ok.best_params_

{'clf-svm__alpha': 1e-05,
 'tfidf__use_idf': True,
 'vect__max_df': 0.75,
 'vect__ngram_range': (1, 2)}

In [127]:
predicted

array([10, 16, 10, ...,  3,  3,  7])

In [128]:
data_test.target

array([10, 16, 14, ...,  4,  6,  7])

In [129]:
predicted == data_test.target

array([ True,  True, False, ..., False, False,  True])

In [130]:
np.mean([10,9,10])

9.666666666666666

In [131]:
from sklearn.metrics import accuracy_score

In [132]:
print(accuracy_score(data_test.target, predicted))

0.8345724907063197


In [133]:
gs_clf_svm_ok.predict([texto1])

array([4])

In [134]:
predicted = gs_clf_svm_ok.predict(data_test.data)
print('Taxa de acerto: ', np.mean(predicted == data_test.target))

Taxa de acerto:  0.8459904407859798


Montar modelo com melhores parâmetros

In [136]:
text_clf_svm = Pipeline([('vect', CountVectorizer(stop_words='english', max_df=0.75, ngram_range=(1, 2))),
                     ('tfidf', TfidfTransformer(use_idf=False)),
                     ('clf-svm', SGDClassifier(loss='squared_hinge', penalty='elasticnet', alpha=1e-5, max_iter=90, random_state=42)),
])
text_clf_svm = text_clf_svm.fit(data_train.data, data_train.target)
predicted_svm = text_clf_svm.predict(data_test.data)
np.mean(predicted_svm == data_test.target)

0.7643388210302708

In [137]:
data_test.target

array([10, 16, 14, ...,  4,  6,  7])

In [138]:
gs_clf.predict(data_test.data)

array([10, 16, 10, ...,  3,  3,  7])

In [139]:
gs_clf.predict(["baseball baseball baseball"])

array([9])

In [140]:
le = preprocessing.LabelEncoder()

In [141]:
le

LabelEncoder()

In [142]:
data_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [143]:
le.fit(data_train.target_names)

LabelEncoder()

In [144]:
joblib.dump(le, '/Users/gustavorsampaio/Documents/Cursos/Python 2023.2/predictor_label.pkl')

['/Users/gustavorsampaio/Documents/Cursos/Python 2023.2/predictor_label.pkl']

In [145]:
joblib.dump(gs_clf_ok, '/Users/gustavorsampaio/Documents/Cursos/Python 2023.2/predictor_model.pkl')

['/Users/gustavorsampaio/Documents/Cursos/Python 2023.2/predictor_model.pkl']

# Prever dados Não Classificados:

In [146]:
from sklearn.preprocessing import LabelEncoder
import joblib

grs_clf = joblib.load('/Users/gustavorsampaio/Documents/Cursos/Python 2023.2/predictor_model.pkl')
grs_le = joblib.load('/Users/gustavorsampaio/Documents/Cursos/Python 2023.2/predictor_label.pkl')

In [147]:
grs_le

LabelEncoder()

In [148]:
grs_clf

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(stop_words='english')),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultinomialNB(fit_prior=False))]),
             n_jobs=-1,
             param_grid={'clf__alpha': (0.01, 1e-05),
                         'tfidf__use_idf': (True, False),
                         'vect__max_df': (0.75, 0.8),
                         'vect__ngram_range': [(1, 1), (1, 2), (1, 3)]})

### Colocar os textos numa lista para passar ao classificador.

In [155]:
#p = data_test.data[0:10]
p = ['My computer runs faster than yours. It has 96 gb of RAM and it is a Macbook Pro. Macs ate the best']

In [156]:
p

['My computer runs faster than yours. It has 96 gb of RAM and it is a Macbook Pro. Macs ate the best']

In [157]:
predicted_model = grs_clf.predict(p)

In [158]:
predicted_model

array([4])

In [159]:
predicted_labels = grs_le.inverse_transform(predicted_model)

In [160]:
print(predicted_labels)

['comp.sys.mac.hardware']


In [161]:
teste1 = 'My computer runs faster than yours. It has 96 gb of RAM and it is a Macbook Pro. Macs ate the best'

In [162]:
teste2 = "CLEVELAND -- Guardians catching prospect Andrés Meléndez died suddenly on Thursday afternoon in Miami, the team announced. He was 20 years old. No details were released on the cause of his death. Meléndez was born in Barquisimeto, Venezuela, and began his professional career in 2018, with the Brewers. He was acquired by Cleveland in a November '19 trade with Milwaukee. In 2021, he played 73 games for Low-A Lynchburg. In two years with the Guardians' organization, Meléndez quickly made an impact on his teammates, both in Lynchburg and the Arizona Rookie League."

In [163]:
predicted_model = grs_clf.predict([teste1, teste2])

In [164]:
predicted_labels = grs_le.inverse_transform(predicted_model)

In [165]:
print(predicted_labels)

['comp.sys.mac.hardware' 'rec.sport.baseball']
